In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, MultiPolygon
import osmnx as ox

from sklearn import linear_model
import statsmodels.api as sm

/Users/lamderek/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/Users/lamderek/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/lamderek/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


directly skip to statsmodel

In [2]:
temp_trial = gpd.read_file("/Users/lamderek/Documents/Sudo/Study/University Study/Year 4/RECO4001 Dissertation/02_Analysis/01_resources/backupbackup.geojson")

DriverError: /Users/lamderek/Documents/Sudo/Study/University Study/Year 4/RECO4001 Dissertation/02_Analysis/01_resources/backupbackup.geojson: No such file or directory

In [8]:
temp_trial.info()
temp_trial.sample(3)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   DCODE        449 non-null    object  
 1   EV_COUNT     67 non-null     float64 
 2   MPS_COUNT    114 non-null    float64 
 3   BLDG_COUNT   347 non-null    float64 
 4   MA_HH        448 non-null    float64 
 5   EDU_PERCENT  448 non-null    float64 
 6   T_POP        448 non-null    float64 
 7   T_WP         448 non-null    float64 
 8   geometry     449 non-null    geometry
dtypes: float64(7), geometry(1), object(1)
memory usage: 31.7+ KB


,DCODE,EV_COUNT,MPS_COUNT,BLDG_COUNT,MA_HH,EDU_PERCENT,T_POP,T_WP,geometry
435,Q27,NaN,NaN,10.0,40890.000000,0.245822,3498.0,1338.0,"POLYGON ((854976.00000 833661.00000, 856976.00..."
7,F1,NaN,NaN,422.0,15116.666667,0.194931,4979.0,2088.0,"POLYGON ((802976.00000 811661.00000, 804976.00..."
88,D8,NaN,NaN,19.0,28100.000000,0.287173,1567.0,871.0,"POLYGON ((816976.00000 807661.00000, 818976.00..."


In [9]:
temp_trial["EV_COUNT"].fillna(0, inplace = True)
temp_trial["MPS_COUNT"].fillna(0, inplace = True)
temp_trial["BLDG_COUNT"].fillna(0, inplace = True)

temp_trial["MA_HH"].fillna(temp_trial["MA_HH"].mean(), inplace = True)
temp_trial["EDU_PERCENT"].fillna(temp_trial["EDU_PERCENT"].mean(), inplace = True)
temp_trial["T_POP"].fillna(temp_trial["T_POP"].mean(), inplace = True)
temp_trial["T_WP"].fillna(temp_trial["T_WP"].mean(), inplace = True)

In [10]:
# temp_trial["EDU"] = temp_trial["EDU_PERCENT"]*temp_trial['T_POP']
# temp_trial.drop("EDU_PERCENT", 1, inplace = True)
# temp_trial["EDU"] = temp_trial["EDU"]/1000

temp_trial["MA_HH"] = temp_trial["MA_HH"]/100
temp_trial["T_POP"] = temp_trial["T_POP"]/1000
temp_trial["T_WP"] = temp_trial["T_WP"]/1000


In [69]:
temp_trial.sample(5)

,DCODE,EV_COUNT,MPS_COUNT,BLDG_COUNT,MA_HH,T_POP,T_WP,geometry,EDU
18,A3,0.0,0.0,10.0,227.500000,1.696,0.600,"POLYGON ((806976.00000 801661.00000, 808976.00...",0.457000
443,O28,0.0,0.0,0.0,517.800000,1.468,0.335,"POLYGON ((856976.00000 829661.00000, 858976.00...",0.324000
391,O24,0.0,0.0,0.0,409.266667,5.504,2.469,"POLYGON ((848976.00000 829661.00000, 850976.00...",1.639163
180,V13,0.0,0.0,0.0,155.000000,2.208,0.982,"POLYGON ((826976.00000 843661.00000, 828976.00...",0.331000
401,L25,0.0,0.0,1.0,517.800000,1.468,0.335,"POLYGON ((850976.00000 823661.00000, 852976.00...",0.324000


In [11]:
reg_data = pd.DataFrame(temp_trial.drop(["geometry"], 1))
reg_data.to_csv("/Users/lamderek/Documents/Sudo/Study/University Study/Year 4/RECO4001 Dissertation/02_Analysis/01_resources/reg_data_v1.csv")
reg_data.sample(5)

<ipython-input-11-f7728a0ed3f8>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  reg_data = pd.DataFrame(temp_trial.drop(["geometry"], 1))


,DCODE,EV_COUNT,MPS_COUNT,BLDG_COUNT,MA_HH,EDU_PERCENT,T_POP,T_WP
126,I10,0.0,0.0,227.0,600.000000,0.580199,7.656,4.077
103,C9,0.0,0.0,81.0,199.280000,0.190190,16.395,8.117
90,F8,0.0,0.0,10.0,224.766667,0.296334,7.447,3.545
402,M25,0.0,0.0,12.0,463.900000,0.311252,3.474,1.466
96,M8,0.0,86.0,228.0,353.766667,0.355697,14.373,6.894


### Scikit-Learn

In [26]:
dependent_vars = ['MPS_COUNT', 'BLDG_COUNT', 'MA_HH', 'EDU_PERCENT', 'T_POP', 'T_WP']

X = temp_trial[dependent_vars]
y = temp_trial.EV_COUNT

# print(X.shape)
# print(y.shape)
clf = linear_model.PoissonRegressor()
clf.fit(X, y)

/Users/lamderek/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_glm/link.py:90: RuntimeWarning: overflow encountered in exp
  return np.exp(lin_pred)
/Users/lamderek/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_glm/link.py:93: RuntimeWarning: overflow encountered in exp
  return np.exp(lin_pred)
/Users/lamderek/anaconda3/lib/python3.8/site-packages/sklearn/_loss/glm_distribution.py:132: RuntimeWarning: invalid value encountered in true_divide
  return -2 * (y - y_pred) / self.unit_variance(y_pred)
/Users/lamderek/anaconda3/lib/python3.8/site-packages/sklearn/_loss/glm_distribution.py:323: RuntimeWarning: invalid value encountered in add
  dev = 2 * (xlogy(y, y / y_pred) - y + y_pred)
/Users/lamderek/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_glm/glm.py:323: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-lea

PoissonRegressor()

In [33]:
clf.score(X, y)

0.0

In [ ]:
mod = sm.OLS(y, X) 
res = mod.fit()  
res.summary()

### statsmodel

In [3]:
reg_data = pd.read_csv("/Users/lamderek/Documents/Sudo/Study/University Study/Year 4/RECO4001 Dissertation/02_Quantative Analysis/01_resources/reg_data_v1.csv")

In [4]:
import numpy as np
import pandas as pd
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable,
    Independence,Autoregressive)
from statsmodels.genmod.families import Poisson

fam = Poisson()
ind = Independence()
model1 = GEE.from_formula("EV_COUNT ~ T_POP + BLDG_COUNT + MA_HH + EDU_PERCENT +  MPS_COUNT", 
                         groups = "EV_COUNT", data = reg_data, cov_struct=ind, family=fam)
result1 = model1.fit()
print(result1.summary())


                               GEE Regression Results                              
Dep. Variable:                    EV_COUNT   No. Observations:                  449
Model:                                 GEE   No. clusters:                       13
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                 382
Family:                            Poisson   Mean cluster size:                34.5
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Thu, 14 Apr 2022   Scale:                           1.000
Covariance type:                    robust   Time:                         13:11:45
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -4.2160      1.232     -3.422      0.001      -6.631      -1.801
T_PO

In [48]:
# import statsmodels.api as sm
# import statsmodels.formula.api as smf
# # data = sm.datasets.get_rdataset('epil', package='MASS').data

# fam = sm.families.Poisson()
# ind = sm.cov_struct.Exchangeable()
# mod = smf.poisson(formula= "EV_COUNT ~T_POP + EDU_PERCENT  + MA_HH +  MPS_COUNT + BLDG_COUNT", data = temp_trial)
# res = mod.fit()
# print(res.summary())

,EV_COUNT,MPS_COUNT,BLDG_COUNT,MA_HH,EDU_PERCENT,T_POP,T_WP
EV_COUNT,1.000000,0.567468,0.601988,0.001433,0.094512,0.730343,0.735994
MPS_COUNT,0.567468,1.000000,0.818452,0.015517,0.075407,0.634176,0.639513
BLDG_COUNT,0.601988,0.818452,1.000000,0.032539,0.117012,0.691868,0.700177
MA_HH,0.001433,0.015517,0.032539,1.000000,0.595716,-0.017436,-0.003607
EDU_PERCENT,0.094512,0.075407,0.117012,0.595716,1.000000,0.057656,0.078374
T_POP,0.730343,0.634176,0.691868,-0.017436,0.057656,1.000000,0.998668
T_WP,0.735994,0.639513,0.700177,-0.003607,0.078374,0.998668,1.000000


In [34]:
reg_data

,DCODE,EV_COUNT,MPS_COUNT,BLDG_COUNT,MA_HH,EDU_PERCENT,T_POP,T_WP
0,C0,0.0,0.0,17.0,22750.0,0.269458,1696.0,600.0
1,D0,0.0,0.0,2.0,22750.0,0.269458,1696.0,600.0
2,E0,0.0,0.0,1.0,22750.0,0.269458,1696.0,600.0
3,F0,0.0,0.0,4.0,16375.0,0.200391,3653.0,1505.0
4,C1,0.0,0.0,0.0,22750.0,0.269458,1696.0,600.0
...,...,...,...,...,...,...,...,...
444,P28,0.0,0.0,0.0,51780.0,0.220708,1468.0,335.0
445,O29,0.0,0.0,0.0,51780.0,0.220708,1468.0,335.0
446,P29,0.0,0.0,0.0,51780.0,0.220708,1468.0,335.0
447,R30,0.0,0.0,1.0,30000.0,0.270936,2030.0,1003.0
